# The Battle of Neighborhoods

## IBM Data Science Professional Certificate - Class #9 Applied Data Science Capstone

### Yuwei Bao 6/28/2020

### Project Title: Recommend a cafe to visitors that is within 200 meters near Bourbon street, New Orleans, Louisiana, U.S.

### Introduction

As a person who is moving to New Orleans soon, I decided to choose a project that is personally related. 

This project will provide visitors the suggestion for a nearby cafe to grab food before they head to Bourbon street, which is the most famous bar street in French Quarter.

This is a really basic project based on the technics. However, for anyone visits a city for the first time and for a short stay, having a recommended cafe will help increase the quality of the visiting.

### Data and References

Data:

- Address data for Bourbon street https://www.mapquest.com/us/louisiana/bourbon-street-265728386, which is used to get the latitude and longitude data.
- Bourbon Street's latitude and longitude coordinates, which is used to get the venue data and to plot the map

References:

- Week 3: Foursquare API lab, which can be downloaded at http://cocl.us/DP0701EN_Coursera_Week2_Notebook1

### Methodology

Foursquare was used to provide the latitude, longitude, name, address, distance, and rating data of the cafe. 

This project processes the rating data, visualizes locations on map, and makes analysis to provide visitors a short report to help they make their decisions.

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [13]:
# This part is the information about my CLEINT_ID, CLIENT_SECRET and other private information. I have removed it for my privacy.

## Before went to the bar at Bourbon street, tourists may want to grab some food

### Converting the Bourbon street's address to its latitude and longitude coordinates
Address data was obtained from https://www.mapquest.com/us/louisiana/bourbon-street-265728386

In [4]:
address = '700 Bourbon St., New Orleans, LA'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

29.9586069 -90.0654573


### Search for cafe that is within 200 meters from Bourbon street

In [5]:
search_query = 'cafe'
radius = 300
print('Searching for '+ search_query)

Searching for cafe


In [6]:
# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5PM0YDTXEDFRWVXI5HYXDU3HL3HVYTYFXU045ESX51MEJ2N3&client_secret=0E55LKYOM31UJQVZV4VJATAGWTNBOQEAQC4E0NSK5R1NO0JH&ll=29.9586069,-90.0654573&v=20180604&query=cafe&radius=300&limit=30'

In [7]:
# Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef940194428846658f32db3'},
 'response': {'venues': [{'id': '4aa59477f964a520dd4820e3',
    'name': 'Café du Monde',
    'location': {'address': '800 Decatur St',
     'crossStreet': 'at The French Market',
     'lat': 29.957509521972145,
     'lng': -90.06205604352482,
     'labeledLatLngs': [{'label': 'display',
       'lat': 29.957509521972145,
       'lng': -90.06205604352482}],
     'distance': 350,
     'postalCode': '70116',
     'cc': 'US',
     'neighborhood': 'French Quarter',
     'city': 'New Orleans',
     'state': 'LA',
     'country': 'United States',
     'formattedAddress': ['800 Decatur St (at The French Market)',
      'New Orleans, LA 70116',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'r

In [8]:
# Get relevant part of JSON and transform it into a pandas dataframe

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4aa59477f964a520dd4820e3,800 Decatur St,...,"[800 Decatur St (at The French Market), New Or...","[{'label': 'display', 'lat': 29.95750952197214...",29.957510,-90.062056,French Quarter,70116,LA,Café du Monde,v-1593393413,NaN
1,"[{'id': '4bf58dd8d48988d1d8941735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b7648f6f964a520a8462ee3,901 Bourbon St,...,"[901 Bourbon St (at Dumaine St.), New Orleans,...","[{'label': 'display', 'lat': 29.96030783390194...",29.960308,-90.064120,NaN,70116,LA,Cafe Lafitte In Exile,v-1593393413,48898359
2,"[{'id': '4bf58dd8d48988d17a941735', 'name': 'C...",1334504,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/cafe-masper...,False,4aecf62ef964a52076cc21e3,601 Decatur St,...,"[601 Decatur St (at Toulouse St), New Orleans,...","[{'label': 'display', 'lat': 29.95600468066164...",29.956005,-90.063461,NaN,70130,LA,Cafe Maspero,v-1593393413,48167065
3,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4aeb317af964a5207abf21e3,622 Pirates Aly,...,"[622 Pirates Aly (Cabildo Alley), New Orleans,...","[{'label': 'display', 'lat': 29.95802788039143...",29.958028,-90.064133,NaN,70116,LA,Tony Seville's Pirates Alley Cafe & Old Absint...,v-1593393413,NaN
4,"[{'id': '4bf58dd8d48988d17a941735', 'name': 'C...",1493868,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/cafe-pontal...,False,4b2e6845f964a5209fdf24e3,546 Saint Peter St,...,"[546 Saint Peter St (at Chartres St.), New Orl...","[{'label': 'display', 'lat': 29.957258, 'lng':...",29.957258,-90.063818,NaN,70116,LA,Cafe Pontalba,v-1593393413,NaN


### Define information of interest and filter dataframe

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Café du Monde,Café,800 Decatur St,US,New Orleans,United States,at The French Market,350,"[800 Decatur St (at The French Market), New Or...","[{'label': 'display', 'lat': 29.95750952197214...",29.957510,-90.062056,French Quarter,70116,LA,4aa59477f964a520dd4820e3
1,Cafe Lafitte In Exile,Gay Bar,901 Bourbon St,US,New Orleans,United States,at Dumaine St.,229,"[901 Bourbon St (at Dumaine St.), New Orleans,...","[{'label': 'display', 'lat': 29.96030783390194...",29.960308,-90.064120,NaN,70116,LA,4b7648f6f964a520a8462ee3
2,Cafe Maspero,Cajun / Creole Restaurant,601 Decatur St,US,New Orleans,United States,at Toulouse St,347,"[601 Decatur St (at Toulouse St), New Orleans,...","[{'label': 'display', 'lat': 29.95600468066164...",29.956005,-90.063461,NaN,70130,LA,4aecf62ef964a52076cc21e3
3,Tony Seville's Pirates Alley Cafe & Old Absint...,Bar,622 Pirates Aly,US,New Orleans,United States,Cabildo Alley,143,"[622 Pirates Aly (Cabildo Alley), New Orleans,...","[{'label': 'display', 'lat': 29.95802788039143...",29.958028,-90.064133,NaN,70116,LA,4aeb317af964a5207abf21e3
4,Cafe Pontalba,Cajun / Creole Restaurant,546 Saint Peter St,US,New Orleans,United States,at Chartres St.,218,"[546 Saint Peter St (at Chartres St.), New Orl...","[{'label': 'display', 'lat': 29.957258, 'lng':...",29.957258,-90.063818,NaN,70116,LA,4b2e6845f964a5209fdf24e3
5,Cafe Beignet,Café,334 Royal St Ste B,US,New Orleans,United States,btwn Conti St & Bienville St,398,[334 Royal St Ste B (btwn Conti St & Bienville...,"[{'label': 'display', 'lat': 29.95533280047758...",29.955333,-90.067120,NaN,70130,LA,4ad4c04df964a52072f320e3
6,Cafe Soule and The Paris Room,Cajun / Creole Restaurant,720 Saint Louis St,US,New Orleans,United States,Bourbon & Royal,236,"[720 Saint Louis St (Bourbon & Royal), New Orl...","[{'label': 'display', 'lat': 29.95673605598335...",29.956736,-90.066626,NaN,70130,LA,4ef3f22b754a89ace0027eb6
7,Cafe Du Monde: Wholesale,Gift Shop,1039 Decatur St,US,New Orleans,United States,NaN,369,"[1039 Decatur St, New Orleans, LA 70116, Unite...","[{'label': 'display', 'lat': 29.95751655637996...",29.957517,-90.061835,NaN,70116,LA,4bae2d4ff964a5200b8e3be3
8,Cafe Beignet at the Old Coffee Pot,Café,714 Saint Peter St,US,New Orleans,United States,NaN,70,"[714 Saint Peter St, New Orleans, LA 70116, Un...","[{'label': 'display', 'lat': 29.958095, 'lng':...",29.958095,-90.065028,NaN,70116,LA,5e50195087d44900083aa068
9,Vacherie Cafe,Café,827 Toulouse St,US,New Orleans,United States,at Dauphine St.,156,"[827 Toulouse St (at Dauphine St.), New Orlean...","[{'label': 'display', 'lat': 29.95841466673428...",29.958415,-90.067066,NaN,70112,LA,4e7601a4d22d80eb33729adf


In [10]:
# Just gonna save these results to a csv file for backup
dataframe_filtered.to_csv('cafe_200_near_BourbonStreet_data')

In [11]:
# Output just the names of these cafe
dataframe_filtered.name

0                                         Café du Monde
1                                 Cafe Lafitte In Exile
2                                          Cafe Maspero
3     Tony Seville's Pirates Alley Cafe & Old Absint...
4                                         Cafe Pontalba
5                                          Cafe Beignet
6                         Cafe Soule and The Paris Room
7                              Cafe Du Monde: Wholesale
8                    Cafe Beignet at the Old Coffee Pot
9                                         Vacherie Cafe
10                                          Café Amelie
11                            Internet Cafe On Toulouse
12                                         Cafe Beignet
13                                           Cafe Opera
14                                            Cane Cafe
15                                          Cafe Iguana
16                                        Cafè Dauphine
17                                         Cafe 

### Visualize these cafes that are nearby

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=17) # generate map centred around Bourbon Street

# add a red circle marker to represent the Bourbon Street
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Bourbon Street',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the cafes as black circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Explore a Given Venue

#### 1. Start with the closest cafe -- Cafe Beignet at the Old Coffee Pot (70 meters) to get the venue's overall rating

In [33]:
venue_id = '5e50195087d44900083aa068'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5e50195087d44900083aa068?client_id=5PM0YDTXEDFRWVXI5HYXDU3HL3HVYTYFXU045ESX51MEJ2N3&client_secret=0E55LKYOM31UJQVZV4VJATAGWTNBOQEAQC4E0NSK5R1NO0JH&v=20180604'

In [34]:
# Send GET request for result
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'delivery', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '5e50195087d44900083aa068',
 'name': 'Cafe Beignet at the Old Coffee Pot',
 'contact': {'phone': '5045004370', 'formattedPhone': '(504) 500-4370'},
 'location': {'address': '714 Saint Peter St',
  'lat': 29.958095,
  'lng': -90.065028,
  'labeledLatLngs': [{'label': 'display', 'lat': 29.958095, 'lng': -90.065028},
   {'label': 'entrance', 'lat': 29.958195, 'lng': -90.065133}],
  'postalCode': '70116',
  'cc': 'US',
  'city': 'New Orleans',
  'state': 'LA',
  'country': 'United States',
  'formattedAddress': ['714 Saint Peter St',
   'New Orleans, LA 70116',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/cafe-beignet-at-the-old-coffee-pot/5e50195087d44900083aa068',
 'categories': [{'id': '4bf58dd8d48988d16d941735',
   'name': 'Café',
   'pluralName': 'Cafés',
   'shortName': 'Café',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'likes': {'cou

In [35]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Arlight, since this restaurant has no ratings, we will look at the second one

#### 2. The second closest cafe -- Internet Cafe On Toulouse (78 meters)

In [36]:
venue_id = '4f32433619836c91c7c643c6'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Well...no ratings, let's move on

#### 3. The third closest cafe -- Cane Cafe (134 meters)

In [37]:
venue_id = '5b5274aacabcff002cd0ad56'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Nope, let's move on

#### 4. The fourth closest cafe -- Tony Seville's Pirates Alley Cafe & Old Absinthe House (143 meters)

In [39]:
venue_id = '4aeb317af964a5207abf21e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.7


Yay! Finally we got a ranking! Thank God...Since we only have one ranking so far, it is no comparison that tells how good/bad the score 7.7 stands for. So let's move on.

#### 5. The fifth closest cafe -- Internet Café (150 meters)

In [40]:
venue_id = '4ccf428f3bd6236aef748421'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Too bad...Let's move on

#### 6. The sixth closest cafe -- cafedumonde (152 meters)

In [41]:
venue_id = '4f639794e4b0777dfb9086d9'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Nope...moving on...

#### 7. The seventh closest cafe -- Vacherie Cafe (156 meters)

In [42]:
venue_id = '4e7601a4d22d80eb33729adf'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


Yay! Very excited to see another ranking score. So the ranking of this one #7-7.8 is a little bit higher than #4-7.7. Let's check more cafes.

#### 8. The eighth closest cafe -- Cafe Opera (165 meters)

In [43]:
venue_id = '5506c5a6498e18fb132e132f'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Fine. Next one!

#### 9. The ninth closest cafe -- Fiorella's Cafe (173 meters)

In [44]:
venue_id = '4efa375d9911b01dddf33d5a'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Nope. Next.

#### 10. The tenth closest cafe -- Jazz "N" Jazz (183 meters)

Okay...I am not sure why this one is counted as a cafe. It belongs to Miscellaneous Shop category. When I google it, it looks like a music store. I believe it's an error somewhere. I went back and checked the searching query and that seems correct. Maybe they provide food during small concerts? Not sure. However, consider about the main purpose of this store, it will not be recommended as a main cafe for users. So let's move on...

#### 11 & 12 & 13 & 14 cafe

11. La Cafe (215 meters) 
12. Cafè Dauphine (222 meters)
13. Cafe A'du Ma (228 meters)
14. Cafe Soule and The Paris Room (236 meters)

As you can tell, these four cafes seem not within 200 meters, which made me run into big confusion.

If they are out of searching raduis, how can they still be listed as results?

If they are not, why they are further?

Is there something wrong with the searching query?

In order to see why this happens, I went back to my reference: Learning FourSquare API with Python and re-run through the notebook. Still cannot answer these questions. But as for the project that is aiming for cafes within 200 meters. These cafes will not be considered.

### Results & Discussions

1. There are totally nine cafes within 200 meters of Bourbon street.

2. Among these nine cafes, seven of them have not been rated on Foursquare and two have been rated.

3. Between these two rated cafes:
        -Tony Seville's Pirates Alley Cafe & Old Absinthe House, which is 143 meters away from Bourbon street, has a rating score of 7.7
        -Vacherie Cafe, which is 156 meters away from Bourbon street, has a rating score of 7.8

Because the rated two cafes do not have distinct differences on either distances or scores, there is not an absolute better one in these two. 

They both worth being recommended for visitors.

### Conclusion

1. If you prefer a rated cafe that is closer to Bourbon street, please visit Tony Seville's Pirates Alley Cafe & Old Absinthe House.
        Address: 622 Pirates Aly (Cabildo Alley), New Orleans, LA 70116
2. If you prefer a rated cafe that has a higher ranking score on Foursquare, please visit Vacherie Cafe.
        Address: 827 Toulouse St (at Dauphine St.), New Orleans, LA 70112